In [2]:
# -- Switch to postgres user

# sudo -u postgres psql

# -- Create database

# CREATE DATABASE restaurant_reviews;

# -- Create database user and grant all previlages

# CREATE USER admin WITH PASSWORD 'admin123';

# GRANT ALL PRIVILEGES ON DATABASE restaurant_reviews TO admin;



# -- Login using the new user:
# psql -U admin -d restaurant_reviews -h localhost


# -- Create the tables


# -- Restaurants table

# CREATE TABLE restaurants (
#     restaurant_id SERIAL PRIMARY KEY,
#     restaurant_name VARCHAR(255) UNIQUE
# );


# -- Reviews table

# CREATE TABLE reviews (
#     review_id SERIAL PRIMARY KEY,
#     restaurant_id INT REFERENCES restaurants(restaurant_id),
#     review_text TEXT,
#     rating INT,
#     review_date DATE,
#     source VARCHAR(50)
# );



# --check the tables

# \d


# -- List all tables

# \dt

# -- Describe a table (check schema)

# \d table_name


# -- Delete all records

# TRUNCATE TABLE table_name RESTART IDENTITY CASCADE;


In [3]:
# pip install psycopg2-binary
import pandas as pd
import psycopg2

In [31]:
# Read CSV files into pandas DataFrames
reviews = pd.read_csv(r"C:\Users\Administrator\Desktop\Omega_Consultancy-wee-2\data\processed\reviews_processed.csv")
apps = pd.read_csv(r"C:\Users\Administrator\Desktop\Omega_Consultancy-wee-2\data\raw\app_info.csv")

In [32]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------

# Establish a connection to PostgreSQL using psycopg2
# You must make sure the database 'restaurant_reviews' is already created
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="admin",
    password="admin123"
)
# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [35]:
import pandas as pd
import psycopg2

# Load the CSV
# reviews = pd.read_csv("data/processed/reviews_processed.csv")  # adjust path

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="admin",          # or your user
    password="admin123"  # update password
)
cursor = conn.cursor()

# Helper function: get bank_id from bank_name
def get_bank_id(bank_name):
    cursor.execute("SELECT bank_id FROM banks WHERE bank_name=%s LIMIT 1", (bank_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        # Optional: create bank if it doesn't exist
        cursor.execute("INSERT INTO banks (bank_name, app_name) VALUES (%s, %s) RETURNING bank_id",
                       (bank_name, "UnknownApp"))
        bank_id = cursor.fetchone()[0]
        conn.commit()
        return bank_id

# Insert reviews
for _, row in reviews.iterrows():
    bank_id = get_bank_id(row["bank_name"])
    cursor.execute("""
        INSERT INTO reviews (bank_id, review_text, rating, review_date, source)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        bank_id,
        row["review_text"],
        row["rating"],
        row["review_date"],
        row["source"]
    ))

conn.commit()
cursor.close()
conn.close()

print("Reviews inserted successfully!")


Reviews inserted successfully!
